In [1]:
import numpy as np
import Orange
import os

In [19]:
import pandas as pd
df = pd.read_csv("test.csv")
display(df)

X = df.drop('y', axis=1).values
print(type(X))
display(X)

y = df['y'].values
print(type(y))
display(y)

,x1,x2,x3,x4,y
0,1,0,0,1,1
1,0,0,1,0,1
2,0,0,1,1,0
3,1,1,1,1,0


<class 'numpy.ndarray'>


array([[1, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [1, 1, 1, 1]])

<class 'numpy.ndarray'>


array([1, 1, 0, 0])

In [58]:
class Feature:
    def __init__(self, feature_type, thresholds=None):
        self.feature_type = feature_type
        self.thresholds = thresholds

class IMLI:    
    def __init__(self, n_clauses = 0, lamda = 3):
        self.n_clauses = n_clauses
        self.lamda = lamda
        
        self.B = []
        self.eta = []
        self.n_features = 0
            
    def _get_id_maxsat_B(self, l, j):
        return l * self.n_features + j + 1
    
    def _get_id_maxsat_eta(self, q):
        return self.n_clauses * self.n_features + q + 1
    
    def _generate_B_eta(self, assignment, n_samples):
        idx = 0
        B = np.zeros((self.n_clauses, self.n_features))
        for l in range(self.n_clauses):
            for j in range(self.n_features):
                B[l][j] = 1 if assignment[idx] > 0 else 0
                idx += 1
        
        eta = np.zeros((n_samples,))
        for q in range(n_samples):
            eta[q] = 1 if assignment[idx] > 0 else 0
            idx += 1
        
        return B, eta        
    
    def _maxsat_solve(self, X, y):
        n_samples = y.shape[0]
        
        fname = "query.wcnf"
        with open(fname, "w") as f:            
            # define parameters            
            unique, counts = np.unique(y, return_counts = True)
            num_occurence = dict(zip(unique, counts))
            
            nbvar = self.n_features * self.n_clauses + n_samples + num_occurence[0] * self.n_clauses
            nbclauses = self.n_features * self.n_clauses \
                        + n_samples \
                        + num_occurence[0] * (1 + self.n_clauses * (self.n_features // 2)) \
                        + num_occurence[1] * self.n_clauses
            
            top = self.n_features * self.n_clauses + n_samples * self.lamda + 1
            
            line = "p wcnf %d %d %d\r\n" % (nbvar, nbclauses, top)
            f.write(line)
            
            # Constraints
            # We change the indexing system from the paper to 0-based indexing
            # Here, we encode each propositional to maxsat index as follows:
            # 1. B_l_j = variable index (l * n_features + j + 1),
            # 2. eta_q = variable index (n_clauses * n_features + q + 1),
            # 3. z_i_j = the rest, the numbering isn't important since
            #            each of them is only used once
            
            # first constraint
            idx = 1
            for l in range(self.n_clauses):
                for j in range(self.n_features):
                    line = "%d %d 0\r\n" % (1, -(idx))
                    f.write(line)
                    idx += 1 # idx = l * n_features + j + 1
            
            # second constraint
            for q in range(n_samples):
                line = "%d %d 0\r\n" % (self.lamda, -(idx))
                f.write(line)
                idx += 1 # idx = (n_clauses * n_features) + q + 1
            
            # third constraint
            for q in range(n_samples):
                id_eta_q = self._get_id_maxsat_eta(q)
                X_q = X[q]
                on_features = [i for i in range(X_q.shape[0]) if X_q[i] == 1]
                if y[q] == 0:
                    id_z_q_1 = idx
                    
                    # D_q_0
                    clause = [id_eta_q] + [id_z_q_1 + l for l in range(self.n_clauses)]
                    line = str(top) + ' ' + ' '.join(str(u) for u in clause) + ' 0\r\n'
                    f.write(line)
                    
                    # D_q_l
                    for l in range(self.n_clauses):
                        id_z_q_l = id_z_q_1 + l
                        for u in on_features:
                            clause = [-id_z_q_l, -self._get_id_maxsat_B(l, u)]
                            line = str(top) + ' ' + ' '.join(str(u) for u in clause) + ' 0\r\n'
                            f.write(line)
                        
                    idx += self.n_clauses
                        
                elif y[q] == 1:
                    for l in range(self.n_clauses):
                        clause = [id_eta_q] + [l * self.n_features + u + 1 for u in on_features]
                        line = str(top) + ' ' + ' '.join(str(u) for u in clause) + ' 0\r\n'
                        f.write(line)
            
        print("GENERATE SELESAI")
        
        # maxSAT
        print("MAXSAT MULAI")
        os.system("open-wbo ./query.wcnf > result.txt")
        print("MAXSAT SELESAI")
        
        # get result
        rname = "result.txt"
        with open("result.txt", "r") as f:
            lines = f.readlines()
            for line in lines:
                if not line or line[0] == 'c':
                    continue
                elif line[0] == 'v':
                    assignment = [int(u) for u in line.split(' ')[1:-1] ]
                    self.B, self.eta = self._generate_B_eta(assignment, n_samples)
    
    def _is_binary_array(self, x):
        is_binary = True
        for u in x:
            if u != 0 and u != 1:
                is_binary = False
                break
        return is_binary
    
    def _append_X(self, feature_type, new_X, X, id_col, headers=[], thresholds=None):
        if feature_type == "continuous":
            for sign in (">=", "<"):
                for point in thresholds:
                    if sign == ">=":
                        new_col = (np.expand_dims(X[:, id_col], axis=1) >= point).astype(np.int)
                        new_X = np.append(new_X, new_col, axis=1)
                        headers.append("(x%d >= %f)" % (id_col, point))
                    elif sign == "<":
                        new_col = (np.expand_dims(X[:, id_col], axis=1) < point).astype(np.int)
                        new_X = np.append(new_X, new_col, axis=1)
                        headers.append("(x%d < %f)" % (id_col, point))
                        
        elif feature_type == "binary":
            new_col = np.expand_dims(X[:, id_col], axis=1)
        
            new_X = np.append(new_X, new_col, axis=1)
            headers.append("x%d" % (id_col))

            new_X = np.append(new_X, 1 - new_col, axis=1)
            headers.append("(NOT x%d)" % (id_col))
        else:
            raise Error
        
        return new_X
    
    def _preprocess_data(self, X):
        """
        Discretize continuous features and extend with its negation for already binary features
        Also make sure every features already have its negation in the data
        """
        new_X = np.zeros((X.shape[0], 0))
        headers = []
        features = []
        
        disc = Orange.preprocess.discretize.EntropyMDL()
        domain = Orange.data.Domain.from_numpy(X, y)
        orange_table = Orange.data.Table.from_numpy(domain, X, y)
        
        for i, attr in enumerate(orange_table.domain.attributes):
            if self._is_binary_array(X[:, i]):
                new_X = self._append_X("binary", new_X, X, i, headers)
                features.append(Feature("binary"))
            else:
                disc_attr = disc(orange_table, attr)
                thresholds = disc_attr.compute_value.points
                new_X = self._append_X("continuous", new_X, X, i, headers, thresholds)
                features.append(Feature("continuous", thresholds))

        return new_X, headers, features
    
    def _preprocess_test(self, X):
        assert(X.shape[1] == len(self.features))
        new_X = np.zeros((X.shape[0], 0))
        for i, feature in enumerate(self.features):
            if feature.feature_type == "binary":
                new_X = self._append_X("binary", new_X, X, i)
            elif feature.feature_type == "continuous":
                new_X = self._append_X("continuous", new_X, X, i, thresholds=feature.thresholds)
                
        return new_X
    
    def fit(self, X, y):
        """
        Parameters
        ----------
        X : 2D-array of shape (n_samples, n_features)
            Training data
        
        y : 1D-array of shape (n_samples,) with values 0/1
            Target class
        """
        X, headers, features = self._preprocess_data(X)
        n_features = X.shape[1]
        self.n_features = n_features
        self.headers = headers
        self.features = features
        
        self._maxsat_solve(X, y)
    
    def predict(self, X):
        """
        Parameters
        ----------
        X : 2D-array of shape (n_samples, n_features)
            Samples.
        
        Returns
        -------
        C : 1D-array of shape (n_samples,)
            Returns predicted values.
        """
        
        X = self._preprocess_test(X)
        preds = np.matmul(X, self.B.T).prod(axis=1)
        
        return (preds > 0).astype(np.int)
    
    def _clause_to_str(self, clause):
        n_features = self.n_features
        headers = self.headers
        literals = [headers[j] for j in range(n_features) if clause[j] == 1]
        return "[" + " OR ".join(literals) + "]"
        
    def get_rule(self):
        unique_clauses = np.unique(self.B, axis = 0)
        rules_array = [self._clause_to_str(clause) for clause in unique_clauses if clause.size > 0]
        rule = " AND ".join(rules_array)
        return rule
    
imli = IMLI(n_clauses = 1, lamda = 1)
imli.fit(X, y)

GENERATE SELESAI
MAXSAT MULAI
MAXSAT SELESAI


'[(x4 >= 0.041440)]'

In [59]:
preds = imli.predict(X)
print(imli.get_rule())

from sklearn.metrics import classification_report
print(classification_report(preds, y))

[(x4 >= 0.041440)]
              precision    recall  f1-score   support

           0       0.53      1.00      0.69        67
           1       1.00      0.79      0.88       284

    accuracy                           0.83       351
   macro avg       0.77      0.90      0.79       351
weighted avg       0.91      0.83      0.85       351



In [61]:
imli = IMLI(n_clauses = 2, lamda = 2)
imli.fit(X, y)
preds = imli.predict(X)
print(imli.get_rule())

from sklearn.metrics import classification_report
print(classification_report(preds, y))

GENERATE SELESAI
MAXSAT MULAI
MAXSAT SELESAI
[(x26 < 0.999945)] AND [(x4 >= 0.418075) OR (x22 >= 0.186570)]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       129
           1       0.92      0.94      0.93       222

    accuracy                           0.91       351
   macro avg       0.91      0.90      0.90       351
weighted avg       0.91      0.91      0.91       351



In [35]:
import pandas as pd
df = pd.read_csv("ionosphere.csv", header=None)
display(df.head())

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [36]:
import numpy as np
import Orange

df_numpy = df.values
X = df_numpy[:, :-1]
y = np.array([1 if u == 'g' else 0 for u in df_numpy[:, -1]])
domain = Orange.data.Domain.from_numpy(X, y)
ionosphere_table = Orange.data.Table.from_numpy(domain, X, y)

In [37]:
imli = IMLI(n_clauses = 3, lamda = 3)
imli.fit(X, y)

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]], dtype=object)

['x0',
 'x1',
 'x2 >= 0.190280',
 'x2 >= 0.739470',
 'x2 >= 0.998505',
 'x2 < 0.190280',
 'x2 < 0.739470',
 'x2 < 0.998505',
 'x3 >= -0.609635',
 'x3 >= -0.000170',
 'x3 >= 0.007075',
 'x3 >= 0.746850',
 'x3 < -0.609635',
 'x3 < -0.000170',
 'x3 < 0.007075',
 'x3 < 0.746850',
 'x4 >= 0.041440',
 'x4 >= 0.418075',
 'x4 >= 0.995175',
 'x4 < 0.041440',
 'x4 < 0.418075',
 'x4 < 0.995175',
 'x5 >= -0.795310',
 'x5 >= -0.217515',
 'x5 >= -0.000715',
 'x5 >= 0.001010',
 'x5 >= 0.825090',
 'x5 < -0.795310',
 'x5 < -0.217515',
 'x5 < -0.000715',
 'x5 < 0.001010',
 'x5 < 0.825090',
 'x6 >= 0.029375',
 'x6 >= 0.999995',
 'x6 < 0.029375',
 'x6 < 0.999995',
 'x7 >= -0.983575',
 'x7 >= -0.000375',
 'x7 >= 0.000985',
 'x7 >= 0.994515',
 'x7 < -0.983575',
 'x7 < -0.000375',
 'x7 < 0.000985',
 'x7 < 0.994515',
 'x8 >= -0.704705',
 'x8 >= -0.036395',
 'x8 >= 0.011975',
 'x8 >= 0.055610',
 'x8 < -0.704705',
 'x8 < -0.036395',
 'x8 < 0.011975',
 'x8 < 0.055610',
 'x9 >= -0.572990',
 'x9 >= 0.591395',
 'x9

In [33]:
disc = Orange.preprocess.discretize.EntropyMDL()
# disc.method = Orange.preprocess.discretize.EntropyMDL()

disc_ion = disc(ionosphere_table, "Feature 02")
# display(type(disc_ion[:, 0]))
# display(disc_ion[:, 0])
# display(disc_ion.domain)
# display(disc_ion.domain.attributes[32].compute_value.points)
# display(disc_ion)
# display(ionosphere_table.domain.attributes)

In [8]:
def append_X(X, np_table, col, thresholds, columns):
    for sign in (">=", "<"):
        for point in thresholds:
            if sign == ">=":
                new_col = (np.expand_dims(np_table[:, i], axis=1) >= point).astype(np.int)
                X = np.append(X, new_col, axis=1)
                columns.append("x%d >= %f" % (i, point))
            elif sign == "<":
                new_col = (np.expand_dims(np_table[:, i], axis=1) < point).astype(np.int)
                X = np.append(X, new_col, axis=1)
                columns.append("x%d < %f" % (i, point))
                
    return X

disc = Orange.preprocess.discretize.EntropyMDL()
new_X = np.zeros((X.shape[0], 0))
columns = []
for i, attr in enumerate(ionosphere_table.domain.attributes):
    # all var is ContinuousVariable
    disc_attr = disc(ionosphere_table, attr)
    thresholds = disc_attr.compute_value.points
#     new_X = np.append(new_X, np.expand_dims(df_numpy[:, i], axis=1), axis=1)
    new_X = append_X(new_X, df_numpy, i, thresholds, columns)
    
display(new_X)
display(new_X.shape)
display(columns[:10])

array([[1., 0., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       ...,
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.]])

(351, 222)

['x0 >= 0.500000',
 'x0 < 0.500000',
 'x2 >= 0.190280',
 'x2 >= 0.739470',
 'x2 >= 0.998505',
 'x2 < 0.190280',
 'x2 < 0.739470',
 'x2 < 0.998505',
 'x3 >= -0.609635',
 'x3 >= -0.000170']

In [9]:
# imli = IMLI(n_clauses = 2, lamda = 2)
# imli.fit(new_X, y)
# imli.get_rule()

In [10]:
# preds = imli.predict(new_X)

In [11]:
# display(preds)

In [12]:
# from sklearn.metrics import classification_report
# print(classification_report(preds, y))

In [13]:
display(new_X)
display(y)

array([[1., 0., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       ...,
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.]])

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size = 0.2)
display(X_train.shape)
display(X_test.shape)

imli = IMLI(n_clauses = 2, lamda = 2)
imli.fit(X_train, y_train)
print(imli.get_rule())

preds = imli.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(preds, y_test))

(280, 222)

(71, 222)

GENERATE SELESAI
MAXSAT MULAI
MAXSAT SELESAI
[(NOT x20)] AND [x22 OR x148 OR x178]
              precision    recall  f1-score   support

           0       0.75      0.94      0.83        16
           1       0.98      0.91      0.94        55

    accuracy                           0.92        71
   macro avg       0.87      0.92      0.89        71
weighted avg       0.93      0.92      0.92        71

